In [1]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")
import glob
import ee
import pickle
ee.Initialize()


In [2]:

def maskS2clouds(image):

  qa = image.select('QA60');
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask);
  

def NDWI_function(image):
  NDWI = image.expression(
      '(B03-B08)/(B03+B08)', {
        'B03': image.select('B3'),
        'B08': image.select('B8')

  }).rename('NDWI');
  return image.addBands(NDWI);

    
def TBVI1_function(image):

  TBVI1 = image.expression(
    'B05/(B05+B02)', {
      'B02': image.select('B2'),
      'B05': image.select('B5')

  }).rename('TBVI1');
  return image.addBands(TBVI1);


    
def NDVI_G_function(image):

  NDVI_G = image.expression(
  '(B8A-B03)/(B8A+B03)', {
  'B03': image.select('B3'),
  'B8A': image.select('B8A')

  }).rename('NDVI_G');
  return image.addBands(NDVI_G)

    
def SR_N_function(image):
  SR_N = image.expression(
  'B8A/B02', {
  'B02': image.select('B2'),
  'B8A': image.select('B8A')

  }).rename('SR_N');
  return image.addBands(SR_N);

def SR_n2_function(image):
  SR_n2 = image.expression(
      'B08/B03', {
        'B03': image.select('B3'),
        'B08': image.select('B8')
  
  }).rename('SR_n2');
  return image.addBands(SR_n2)

def NDVI_function(image):

  NDVI = image.expression(
      '(B08-B04)/(B08+B04)', {
        'B04': image.select('B4'),
        'B08': image.select('B8')

  }).rename('NDVI');
  return image.addBands(NDVI);

def maskS2clouds_min_max(image):
  
  qa = image.select('QA60');

  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;

  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  img = image.updateMask(mask).divide(10000);
  img_time = img.set("system:time_start", image.get("system:time_start"))
  return img_time

def addNDVI(image):
  ndvi_s = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
  ndvi = ndvi_s.set("system:time_start", image.get("system:time_start"));
  return image.addBands(ndvi)

def clipToCol(image):
  return image.clipToCollection(point)


def masker(image):
  mask1 = greenest.select('NDVI').gt(0.3);
  mask2 = min.select('NDVI').lt(0.6);
  mask3=image.select('NDVI').gt(0.05)
  mask4=image.select('NDVI').lt(0.3)
  return image.updateMask(mask1).updateMask(mask2).updateMask(mask3).updateMask(mask4)



In [153]:
df = pd.read_csv("../lat_lons.csv")

df.loc[:40, 'Date'] = pd.to_datetime(df.loc[:40,'Date'].apply(lambda x: x.split("T")[0]), format = '%Y-%m-%d')

In [155]:
df_tmp = df.loc[:10, :]

In [46]:
df_tmp.columns

Index(['Date', 'Latitude', 'Longitude'], dtype='object')

In [169]:

def masker(greenest, min):
    def wrap(image):                
        mask1 = greenest.select('NDVI').gt(0.3);
        mask2 = min.select('NDVI').lt(0.6);
        mask3=image.select('NDVI').gt(0.05)
        mask4=image.select('NDVI').lt(0.3)
        return image.updateMask(mask1).updateMask(mask2).updateMask(mask3).updateMask(mask4)
    return wrap

def clipToCol(point):
    def wrap(image):
            
        return image.clipToCollection(point)
    return wrap

def predictor_time(row):
    
    current_date = row['Date']

    start_date = ee.Date(current_date - datetime.timedelta(days = 16))

    end_date= ee.Date(current_date)
    point = ee.FeatureCollection(ee.Geometry.Point([row['Longitude'], row['Latitude']]));


    s2_sr_filter = s2_sr.filterBounds(point).filterDate(start_date,end_date).map(maskS2clouds);
                    
    sentinel2 = ee.ImageCollection(s2_sr_filter.mosaic().set("system:time_start", ee.Image(s2_sr_filter.first()).get("system:time_start")))

    indices_collection = sentinel2.map(NDWI_function).map(TBVI1_function).map(NDVI_G_function).map(SR_N_function).map(SR_n2_function).map(NDVI_function);
    

    s2_NDVI = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(point).filterDate('2020-01-01', '2020-12-31')\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 50)).map(maskS2clouds_min_max).map(addNDVI).select("NDVI").map(clipToCol(point));

    greenest = s2_NDVI.qualityMosaic('NDVI');
    min = s2_NDVI.min();

    predictor_bands = indices_collection.map(masker(greenest, min))

    df = pd.DataFrame(predictor_bands.select('B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G')\
        .getRegion(point, 10).getInfo())
    
    df, df.columns = df[1:] , df.iloc[0] 
    df = df.drop(["id"], axis =1)

    return df.values[0]


# asap = df_tmp.apply(predictor_time, axis = 1, result_type='expand', )

In [171]:
df_tmp

,Date,Latitude,Longitude
0,2021-03-03 00:00:00,33.800612,74.934720
1,2021-03-03 00:00:00,33.800849,74.808513
2,2021-03-03 00:00:00,33.800826,74.935571
3,2021-03-03 00:00:00,33.800448,74.935790
4,2021-03-03 00:00:00,33.800858,74.935616
5,2021-03-03 00:00:00,33.800694,74.935908
6,2021-03-03 00:00:00,33.800607,74.935758
7,2021-03-03 00:00:00,33.800885,74.935293
8,2021-03-03 00:00:00,33.801004,74.935320
9,2021-03-03 00:00:00,33.800739,74.936045


In [166]:
df.columns

Index(['Date', 'Latitude', 'Longitude'], dtype='object')

In [176]:
asap = df_tmp.apply(predictor_time, axis = 1,  result_type='expand')
asap.columns = ['longitude', 'latitude', 'time' ,'B8', 'B4', 'B5', 'B11', 'B9', 'B1', 'SR_n2','SR_N', 'TBVI1', 'NDWI', 'NDVI_G']

In [ ]:
input

In [204]:
soil_nuts = [ 'P', 'K', 'OC', 'N', 'pH']

input = asap[['B8', 'B4', 'B5', 'B11', 'B9', 'B1', 'SR_n2','SR_N', 'TBVI1', 'NDWI', 'NDVI_G']]

mylist = []
asd = input.dropna()
for i in soil_nuts:
    
    for nut, nut_slr in zip(glob.glob(f'../data/{i}.pkl'), glob.glob(f'../data/{i}*_slr.pkl')):
        # print(i, nut, nut_slr)
        with open(nut, 'rb') as file, open(nut_slr, 'rb') as file_slr:

            mymodel, model_scaler = pickle.load(file), pickle.load(file_slr)
            X_test = model_scaler.transform(asd)
            
            predictions = mymodel.predict(X_test)
            mylist.append(predictions)
    
# print(pd.DataFrame(mylist))



In [205]:
predictions

array([[7.955 ],
       [7.955 ],
       [8.205 ],
       [6.9125],
       [7.825 ],
       [7.6   ]])

In [112]:
soil_nuts = ['pH', 'P', 'K', 'OC', 'N']
df_tmp = df[["longitude", "latitude"]]

input_bands = ['B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G']
df[input_bands] = df[input_bands].apply(lambda x: x.replace("None", np.nan))


df_pred = df.loc[df['NDWI'].notna(), input_bands]

input = df_pred.values

for i in soil_nuts:
    
  for nut, nut_slr in zip(glob.glob(f'../data/{i}.pkl'), glob.glob(f'../data/{i}*_slr.pkl')):
    # print(i, nut, nut_slr)
    with open(nut, 'rb') as file, open(nut_slr, 'rb') as file_slr:

      mymodel, model_scaler = pickle.load(file), pickle.load(file_slr)
      X_test = model_scaler.transform(input)
      predictions = mymodel.predict(X_test)
      print(predictions, i)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,id,longitude,latitude,time,B8,B4,B5,B11,B9,B1,SR_n2,SR_N,TBVI1,NDWI,NDVI_G,NDVI
1,0,74.934721,33.800595,1613454580078,None,None,None,None,None,None,None,None,None,None,None,None


In [15]:
s2_sr = ee.ImageCollection("COPERNICUS/S2_SR")


current_date = datetime.datetime.strptime("2021-03-03", '%Y-%m-%d')

start_date = ee.Date(current_date - datetime.timedelta(days = 90))


end_date= ee.Date(current_date)

point = ee.FeatureCollection(ee.Geometry.Point([74.9347205, 33.8006117]));


s2_sr_filter = s2_sr.filterBounds(point).filterDate(start_date,end_date).map(maskS2clouds);
                  
sentinel2 = ee.ImageCollection(s2_sr_filter.mosaic().set("system:time_start", ee.Image(s2_sr_filter.first()).get("system:time_start")))

indices_collection = sentinel2.map(NDWI_function).map(TBVI1_function).map(NDVI_G_function).map(SR_N_function).map(SR_n2_function).map(NDVI_function);

s2_NDVI = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(point).filterDate('2020-01-01', '2020-12-31')\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 50)).map(maskS2clouds_min_max).map(addNDVI).select("NDVI").map(clipToCol);

greenest = s2_NDVI.qualityMosaic('NDVI');
min = s2_NDVI.min();

predictor_bands = indices_collection.map(masker)

df = pd.DataFrame(predictor_bands.select('B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G').getRegion(point, 10).getInfo())

# df, df.columns = df[1:] , df.iloc[0] 
# df = df.drop(["id", "time"], axis =1)

# input_bands = ['B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G']
# df[input_bands] = df[input_bands].apply(lambda x: x.replace("None", np.nan))



# myfunc("2021-03-03")
# df_pt.apply(myfunc)


In [6]:
soil_nuts = ['pH', 'P', 'K', 'OC', 'N']
df_tmp = df[["longitude", "latitude"]]
df_pred = df.loc[df['NDWI'].notna(), input_bands]

input = df_pred.values

for i in soil_nuts:
    
  for nut, nut_slr in zip(glob.glob(f'../data/{i}.pkl'), glob.glob(f'../data/{i}*_slr.pkl')):
    # print(i, nut, nut_slr)
    with open(nut, 'rb') as file, open(nut_slr, 'rb') as file_slr:

      mymodel, model_scaler = pickle.load(file), pickle.load(file_slr)
      X_test = model_scaler.transform(input)
      predictions = mymodel.predict(X_test)
      print(predictions, i)

,longitude,latitude,B8,B4,B5,B11,B9,B1,SR_n2,SR_N,TBVI1,NDWI,NDVI_G
1,33.800595,74.934721,None,None,None,None,None,None,None,None,None,None,None
